# Main | LLM to Vocabulary


**Request Documentation**: https://platform.openai.com/docs/api-reference/completions/create

**OpenAI Documentation**: https://platform.openai.com/docs/quickstart?context=python

**Find your keys here**: https://platform.openai.com/api-keys

**Keep an eye on your credit usage**: https://platform.openai.com/usage

## OpenAI API Details and Hyper-Parameters

**Models**: The model you want to use (i.e. "gpt-4-turbo", "gpt-4", "gpt-4o", "gpt-4o-mini" or "gpt-3.5-turbo")

**Message**: The message being sent (the prompt)

**Temperature**: Number between 0 and 1, can be adjusted when calling the OpenAI API to control the randomness of the output generated by the language model. It determines how "creative" or "conservative" the responses are. A temperature of 1.0 means the model generates text with standard randomness, while a lower temperature (closer to 0.0) will make it more deterministic. Suggestion: Set the temperature to a moderate value, around 0.2 to 0.5. This will give you some randomness, mimicking the variability in human input, but still maintain enough accuracy to ensure that the responses are meaningful. **In the case of our work, we held this constant at a value of 0.2 to get slight stochasticity.** 

**Frequency_penalty**: Number between -2 and 2, where higher numbers penalize new tokens based on their frequency to that point in the response. The higher the number, the lower the probability of repetition. This parameter penalizes repeated tokens in the output, making responses more diverse and creative. Suggestion: Use a low frequency penalty (e.g., 0.0 to 0.2) to reduce redundancy but still allow the model to use relevant tokens multiple times where necessary. **In the case of our work, we held this constant at a value of 0.0.**

**Top-p (Nucleus Sampling)**: This parameter determines the probability mass considered for each token in the output. A lower top-p (e.g., 0.7) will cause the model to sample from only the more likely tokens, while a top-p of 1.0 will consider all possible outcomes. Suggestion: Set the top_p to 0.8–0.9 to introduce some controlled randomness while still keeping the majority of the likelihood in more probable tokens, ensuring coherent responses. ~~~~~  Higher top_p values (closer to 1): This will include a broader range of possible tokens, increasing diversity and randomness, making the behaviour more stochastic (i.e., less predictable). A value of top_p = 1 means the model can sample from the full probability distribution, leading to more creative and unpredictable results.
Lower top_p values (closer to 0): This restricts the sampling to a smaller subset of tokens (those with the highest probability), leading to more deterministic and conservative outputs, i.e., less stochastic behavior. For example, setting top_p = 0.1 would mean the model will only consider the most probable 10% of tokens.

## Imports

In [122]:
%pwd

'/Users/liamroy/Documents/Studies/Monash_31194990/PHD/Studies/Study_04/LLM_vocab_optimization/scripts'

In [123]:
# Imports
import sys
import importlib

import numpy as np
import random
import time
random.seed(time.time())

from itertools import product

from openai import OpenAI
client = OpenAI()

from robots_and_modules.helper_functions import llm_prompt_reply
from robots_and_modules.prompt_builder import build_prompt
from robots_and_modules.distance_calculator import calculate_distance


## Define Inputs

In [124]:
## Configuration parameters
attempt_ID = '00'

iteration_quantity = 20
llm_model = "gpt-4o-mini"               # gpt-3.5-turbo     | Use this for dev/testing
                                        # gpt-4 / gpt-4o    | Use this when deployed, more expensive
                                        # gpt-4o-mini       | Lightweight, less expensive than gpt4o
temperature_coefficient = 1.0           # Moderately stochastic @ 0.6 to 1.0
frequency_penalty_coefficient = 1.0     # Lightly penalize repetition @ 0.2
top_p_coefficient=1.0                   # Nucleus sampling for controlled randomness @ 0.85 to 0.6
omission_probability = 0.5

# Default to go1_obj and 'spread' state space
robot_string = 'jackal'     # 'go1' or 'jackal'
state_space = 'spread'      # 'spread' or 'minimized'

printer = "Partial"         # "All", "Partial" or None

### Check Robot Module and State Space

In [125]:
# Check if robot_string and state_space are within the bounds of go1/jackal and spread/minimized
if robot_string not in ['go1', 'jackal']:
    print(f"Error: Invalid robot module specified. Using default: {robot_string}")
if state_space not in ['spread', 'minimized']:
    print(f"Error: Invalid state space specified. Using default: {state_space}")

# Dynamically import the module
robot_module = importlib.import_module(f"robots_and_modules.{robot_string}")

### Select State Space based on Module

In [112]:
# The data in this should be in the format: "State Number: [State Name, State Description]"
# Used for Go1 robot
if robot_string == 'go1':
    if state_space == 'spread':
        set_of_states = [
        "S01: [Waiting for Input, The robot is in standby mode waiting for a command from the user.]",
        "S02: [Analyzing Object, The robot is analyzing a target object in front of it on the ground.]", 
        "S03: [Found Object, The robot has found a target object in front of it on the ground.]",
        "S04: [Needs Help, The robot is experiencing an error and needs help from the user.]",
        "S05: [Confused, The robot is confused and unsure what to do.]",
        "S06: [Unsure, It is unclear as the robot does not appear to be in any of the described states.]"
        ]
        
    elif state_space == 'minimized':
        set_of_states = [
        "S01: [Waiting for Input, The robot is in standby mode waiting for a command from the user.]",
        "S02: [Interacting with Object, The robot is pointing out a target object in front of it on the ground.]", 
        "S03: [Needs Help, The robot is in a confused error staate and needs help from the user.]",
        "S04: [Unsure, It is unclear as the robot does not appear to be in any of the described states.]"
        ]



        
# Used for Jackal robot
elif robot_string == 'jackal':
    if state_space == 'spread':
        set_of_states = [
            "S01: [Processing, The robot is analyzing the request and planning the navigation route.]",
            "S02: [Navigating, The robot is actively navigating toward the destination and does not require assistance.]",
            "S03: [Danger, The robot is signaling for the user's attention due to a detected hazard.]",
            "S04: [Stuck, The robot is signaling for the user's attention as its path is blocked.]",
            "S05: [Accomplished, The robot has successfully reached the requested destination.]",
            "S06: [Unsure, The robot's statr is unclear as it does not appear to be in any of the described states.]"
        ]

    elif state_space == 'minimized':
        set_of_states = [
            "S01: [Progressing, The robot is progressing on the assigned task and does not require assistance.]",
            "S02: [Alert, The robot is signaling for the user's attention due to a possible hazard or blocked path.]",
            "S03: [Accomplished, The robot has successfully reached the requested destination.]",
            "S04: [Unsure, The robot's statr is unclear as it does not appear to be in any of the described states.]"
        ]


state_legend_dict = {}

for state in set_of_states:
    code, rest = state.split(": ", 1)
    name = rest.split(",")[0].strip("[]")
    state_legend_dict[code] = name

print(state_legend_dict)


{'S01': 'Processing', 'S02': 'Navigating', 'S03': 'Danger', 'S04': 'Stuck', 'S05': 'Accomplished', 'S06': 'Unsure'}


### Create Robot Instance | Set Assistant Prompt and Context

In [135]:
llm_assistant_prompt = "You are an expert roboticist and understand how to design communicative expressions for human-robot interaction."

robot_instance = robot_module.Robot(set_of_states)

deployment_context = f"Consider a scenario where you are collaborating with a {robot_instance.form_factor} robot to locate and pick strawberries in a strawberry patch."

## Accuracy Proxy Generator

### Generate the Accuracy Proxy .npy Array

In [115]:
# Initialize error and total counters
error_counter = 0
total_counter = 0

# Get action space shape, and use it to create a for loop that itterates all indices
action_space_shape = robot_instance.get_action_space_shape()

## SWAP || 
# Itterate through all possible actions in action space
count = 0
for indices in product(*(range(dim) for dim in action_space_shape)):
    count += 1
    robot_instance.set_active_parameter(list(indices))

    if printer:
        print(f"Action {count}:")
        print(robot_instance.active_parameters)
        print("\n\n")

## END SWAP ||

# ## SWAP || 
# # Set active parameters based on action space shape for robot_instance
# if robot_string == 'go1':
#     test_value_A = [0, 0, 1, 0, 1, 0]  # Example values within range for each parameter
#     test_value_B = [1, 2, 0, 2, 0, 1]  # Example values within range for each parameter
#     test_value_C = [0, 1, 2, 1, 1, 2]  # Example values within range for each parameter
#     set_set = [test_value_A, test_value_B, test_value_C]

# elif robot_string == 'jackal':
#     test_value_A = [0, 1, 0, 0]
#     test_value_B = [1, 0, 1, 1]
#     test_value_C = [2, 1, 2, 0]
#     set_set = [test_value_A, test_value_B, test_value_C]

# for test_values in set_set:
#     robot_instance.set_active_parameter(test_values)
# ## END SWAP ||


    iteration_counter = 0
    iteration_error_counter = 0

    # Five iterations for loop
    for iteration in range(iteration_quantity):

        if printer:
            print(f"**********************************************************************")
            print(f"ITERATION {iteration+1} FOR {robot_instance.active_parameters}")
            print(f"**********************************************************************")

        # Generate description with test values and omission probability
        raw_description = robot_instance.generate_description(omission_probability)

        # Prepare a promt with context + robot description
        summary_prompt = f"{deployment_context} Summarize the explanation below, focusing on describing the robot's actions in this scenario:\n{raw_description}"
        
        ### SWAP UNCOMMENT TO DISABLE TEST AND ENABLE LLM
        # Summarize the context + robot description using llm 
        # summarized_expression = raw_description
        summarized_expression = llm_prompt_reply(prompt=summary_prompt, 
                                                  client=client, 
                                                  llm_model=llm_model, 
                                                  llm_assistant_prompt=llm_assistant_prompt, 
                                                  temperature_coefficient=temperature_coefficient, 
                                                  frequency_penalty_coefficient=frequency_penalty_coefficient, 
                                                  top_p_coefficient=top_p_coefficient)


        # Print before and after summary
        if printer == "All":
            print(f"\nRAW DESCRIPTION:\n{raw_description}")
            print(f"\nLLM SUMMARY:\n{summarized_expression}")

        # Pass summarized expression to prompt builder, along with set of states and robot_instance parameters 
        acc_proxy_prompt = build_prompt(expression_string=summarized_expression, state_list=set_of_states, deployment_context=deployment_context, llm_assistant_prompt=llm_assistant_prompt)
        
        if printer == "All":
            print(f"\n\nACCURACY PROXY PROMPT:\n~~~{acc_proxy_prompt}~~~\n\n")


        ### SWAP UNCOMMENT TO DISABLE TEST AND ENABLE LLM
        ## Use test data or run the prompt through the accuracy proxy model
        # six_state_test_options = ["[S01, State 01]", "[S02, State 02]", "[S03, State 03]", "[S04, State 04]", "[S05, State 05]", "[S06, State 06]"]
        # four_state_test_options = ["[S01, State 01]", "[S02, State 02]", "[S03, State 03]", "[S04, State 04]"]
        # acc_proxy_reply = random.choice(six_state_test_options)
        acc_proxy_reply = llm_prompt_reply(acc_proxy_prompt, client, llm_model, llm_assistant_prompt, temperature_coefficient, frequency_penalty_coefficient, top_p_coefficient)
        print(f"ACCURACY PROXY REPLY: {acc_proxy_reply}\n")

        # Parse the accuracy proxy reply to identify the state
        try:
            state_code, _ = acc_proxy_reply.strip("[]").split(", ")
            state_code = state_code.strip("'")
        except ValueError:
            print(f"Error: The GPT reply {acc_proxy_reply} was not in the correct format.")
            state_code = "E01"
        
        # Check if the state code exists within the dictionary
        if state_code in robot_instance.action_space[tuple(indices)]:
            # Increment the count for the identified state in the action space
            robot_instance.action_space[tuple(indices)][state_code] += 1
            iteration_counter += 1
            total_counter += 1

            if printer:
                # Output the identified state code
                print(f"Identified state code: {state_code}")
                # Output the updated action space
                print(f"Updated action space: {robot_instance.action_space[tuple(indices)]}\n")

        else:
            # Output an error message if the state code does not exist
            iteration_counter += 1
            iteration_error_counter += 1

            total_counter += 1
            error_counter += 1

            
            if printer:
                print(f"Error: State code '{state_code}' does not exist in the action space.\n")

        #     # Log the reply from the accuracy proxy model to the robot_instance action_space
        #     robot_instance.action_space[1, 1, 0, 0, 1, 0] = acc_proxy_reply
    
    if printer:
        print(f"@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
        print(f"COMPLETED ALL {iteration_quantity} ITERATIONS FOR {robot_instance.active_parameters}\n")
        print(f"Iteration Error Count: {iteration_error_counter}\nIteration Count: {iteration_counter}")
        iteration_error_percentage = (iteration_error_counter / iteration_counter) * 100 if iteration_counter > 0 else 0
        print(f"Iteration Error Percentage: {iteration_error_percentage:.2f}%\n")

        print(f"Total Error Count: {error_counter}\nTotal Count: {total_counter}")
        total_error_percentage = (error_counter / total_counter) * 100 if total_counter > 0 else 0
        print(f"Total Error Percentage: {total_error_percentage:.2f}%")
        print(f"@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\n\n")

if printer:
    print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$")
    print(f"COMPLETED ALL COMBINATIONS")
    print(f"$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$\n\n")

Action 1:
[0 0 0 0]



**********************************************************************
ITERATION 1 FOR [0 0 0 0]
**********************************************************************
ACCURACY PROXY REPLY: [S06, Unsure]

Identified state code: S06
Updated action space: {'S01': 0, 'S02': 3, 'S03': 0, 'S04': 0, 'S05': 0, 'S06': 1}

**********************************************************************
ITERATION 2 FOR [0 0 0 0]
**********************************************************************
ACCURACY PROXY REPLY: [S02, Navigating]

Identified state code: S02
Updated action space: {'S01': 0, 'S02': 4, 'S03': 0, 'S04': 0, 'S05': 0, 'S06': 1}

**********************************************************************
ITERATION 3 FOR [0 0 0 0]
**********************************************************************
ACCURACY PROXY REPLY: [State_Number, State_Name] S02: [Navigating]

Error: State code 'State_Number' does not exist in the action space.

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

### Save RAW Array | Normalize | Save Scaled Array to External File

In [116]:
### Save RAW Array

# Define the save string for the raw numpy array
raw_accuracy_array_filename = robot_string + "_" + state_space + "_acc_array_" + attempt_ID + "_raw.npy"
raw_accuracy_array_save_string = f"./../data/acc_arrays/{raw_accuracy_array_filename}"

# Save the numpy array robot_instance.action_space to a file 
print(f"Saving RAW numpy array to: {raw_accuracy_array_filename}")
np.save(f"{raw_accuracy_array_save_string}", robot_instance.action_space)


### Normalize Array

# Create a new numpy array for the normalized action space
normalized_action_space = np.zeros_like(robot_instance.action_space, dtype=dict)
for index in np.ndindex(robot_instance.action_space.shape):
    dict_obj = robot_instance.action_space[index]
    total = sum(dict_obj.values())  # Compute sum of values
    if total > 0:  # Avoid division by zero
        normalized_action_space[index] = {key: val / total for key, val in dict_obj.items()}


### Save NORMALIZED Array

# Define the save string for the raw numpy array
normalized_accuracy_array_filename = robot_string + "_" + state_space + "_acc_array_" + attempt_ID + "_norm.npy"
normalized_accuracy_array_save_string = f"./../data/acc_arrays/{normalized_accuracy_array_filename}"


# Print the name of the file where the numpy array will be saved
print(f"\nSaving NORMALIZED numpy array to: {normalized_accuracy_array_filename}")
np.save(f"{normalized_accuracy_array_save_string}", normalized_action_space)



# print first element of robot_instance.action_space and normalized_action_space
print("RAW @ [(0, 0, 0, 0)]\t", robot_instance.action_space[(0, 0, 0, 0)])
print("NORM @ [(0, 0, 0, 0)]\t", normalized_action_space[(0, 0, 0, 0)])



Saving RAW numpy array to: jackal_spread_acc_array_00_raw.npy

Saving NORMALIZED numpy array to: jackal_spread_acc_array_00_norm.npy
RAW @ [(0, 0, 0, 0)]	 {'S01': 0, 'S02': 4, 'S03': 0, 'S04': 0, 'S05': 0, 'S06': 1}
NORM @ [(0, 0, 0, 0)]	 {'S01': 0.0, 'S02': 0.8, 'S03': 0.0, 'S04': 0.0, 'S05': 0.0, 'S06': 0.2}


## Generate the Kinematic Distance Object

In [117]:
## Generate external to this file for now. 
## We need to import this file to be used in the following step.

## Evaluate the Cost Function

In [133]:

# Load accuracy proxy values from external file
ACCURACY_PROXY_FILE = normalized_accuracy_array_save_string      # File containing accuracy proxy values   
loaded_accuracy_proxy_array = np.load(ACCURACY_PROXY_FILE, allow_pickle=True) # Load accuracy proxy values

# Define the weight for the cost function
weight = 0.7                                                      # Weight for accuracy vs. distance
distance_method = "emd"                                           # distance metric (none, emd, kinematic)

# Get action space shape and create dict to store final poses
motion_param_ranges = robot_instance.get_action_space_shape()    # Example parameter ranges
poses_dict = {}                                                  # Dictionary to store selected poses


def get_accuracy(A, state_code):
    """
    Retrieves accuracy proxy value for a given pose A.
    """
    return loaded_accuracy_proxy_array[tuple(A)][state_code]  # Access value based on indices in A


def get_min_distance(A):
    """
    Computes the minimum distance between A and all defined expressions in poses_dict.
    """
    if not poses_dict:
        return 0.0
    distances = [calculate_distance(distance_type=distance_method, action_space_shape=motion_param_ranges, np_a=np.array(A), np_b=np.array(B)) for B in poses_dict.values()]
    return min(distances)


def evaluate_cost(A, state_code):
    """
    Evaluates the cost function f(A, B) using state_code.
    """
    accuracy_term = get_accuracy(A, state_code)
    distance_term = get_min_distance(A)
    cost = weight * accuracy_term + (1 - weight) * distance_term
    return cost, accuracy_term, distance_term


def find_best_pose(state_name, state_code):
    """
    Iterates through all possible parameter value combinations to find the best pose.
    """
    best_A = None
    best_cost = float("-inf")
    
    for A in product(*[range(v) for v in motion_param_ranges]):
        cost, accuracy_term, distance_term = evaluate_cost(A, state_code)
        if cost > best_cost:
            best_cost = cost
            best_A = A
            print(f"Pose: {A} | Accuracy: {accuracy_term:.3f} | Distance: {distance_term:.3f} | Cost: {cost:.3f}")
    
    # Store best pose
    if best_A:
        poses_dict[state_name] = best_A
    
    return best_A, best_cost


# Iterate through all states to find the best pose for each state
print(f"\nCalculating Best Poses\nDistance Method: {distance_method}\nWeight: {weight}\n")  

for state_code, state_name in state_legend_dict.items():

    if state_name in poses_dict:
        old_pose = poses_dict[state_name]
        print(f"Overwriting {state_name}: {old_pose} with new pose.")

    best_pose, best_cost = find_best_pose(state_name, state_code)
    print(f"Best Pose for {state_code}: {state_name}: {best_pose} with Cost: {best_cost:.3f}\n")



Calculating Best Poses
Distance Method: emd
Weight: 0.7

Pose: (0, 0, 0, 0) | Accuracy: 0.000 | Distance: 0.000 | Cost: 0.000
Pose: (0, 0, 0, 1) | Accuracy: 0.200 | Distance: 0.000 | Cost: 0.140
Pose: (0, 0, 1, 2) | Accuracy: 0.500 | Distance: 0.000 | Cost: 0.350
Pose: (0, 1, 2, 1) | Accuracy: 0.667 | Distance: 0.000 | Cost: 0.467
Pose: (1, 1, 2, 2) | Accuracy: 1.000 | Distance: 0.000 | Cost: 0.700
Best Pose for S01: Processing: (1, 1, 2, 2) with Cost: 0.700

Pose: (0, 0, 0, 0) | Accuracy: 0.800 | Distance: 0.750 | Cost: 0.785
Pose: (0, 0, 2, 1) | Accuracy: 1.000 | Distance: 0.375 | Cost: 0.812
Pose: (0, 1, 0, 1) | Accuracy: 1.000 | Distance: 0.500 | Cost: 0.850
Pose: (0, 2, 0, 0) | Accuracy: 1.000 | Distance: 0.750 | Cost: 0.925
Best Pose for S02: Navigating: (0, 2, 0, 0) with Cost: 0.925

Pose: (0, 0, 0, 0) | Accuracy: 0.000 | Distance: 0.250 | Cost: 0.075
Pose: (0, 0, 0, 1) | Accuracy: 0.000 | Distance: 0.375 | Cost: 0.113
Pose: (0, 0, 0, 2) | Accuracy: 0.000 | Distance: 0.500 | Co

### Final Printout

In [134]:
print(f"State Legend:")
for key, value in state_legend_dict.items():
    print(f"{key}:{value}")
print()

print("Parameter Legend:")
for i, key in enumerate(robot_instance.parameter_descriptions.keys(), start=1):
    print(f"P{i}: {key}")
print()


print(f"Final Poses:")
for key, value in poses_dict.items():
    print(f"{key}:{value}")
print()


State Legend:
S01:Processing
S02:Navigating
S03:Danger
S04:Stuck
S05:Accomplished
S06:Unsure

Parameter Legend:
P1: Beats Per Loop
P2: Pitch Bend
P3: Gain
P4: Distortion

Final Poses:
Processing:(1, 1, 2, 2)
Navigating:(0, 2, 0, 0)
Danger:(2, 0, 2, 1)
Stuck:(2, 2, 1, 2)
Accomplished:(1, 2, 1, 2)
Unsure:(1, 1, 1, 0)



### Final ToDo:
* Save to external file
